<center>
    This is Set 6: Grouping Methods (Exercises 51-60)
</center>

**Prerequisites**
* The sample dataset [penguins](https://github.com/mwaskom/seaborn-data/blob/master/penguins.csv) from seaborn will be used for the exercises.
* seaborn package

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from seaborn import load_dataset
import pandas as pd

spark = SparkSession.builder.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/28 12:43:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [35]:
pd_data = load_dataset("penguins")
pd_data = pd_data.where(pd.notnull(pd_data), None)

pd_data.dtypes


species               object
island                object
bill_length_mm       float64
bill_depth_mm        float64
flipper_length_mm    float64
body_mass_g          float64
sex                   object
dtype: object

In [36]:
schema = StructType(
    [
        StructField("species", StringType(), True),
        StructField("island", StringType(), True),
        StructField("bill_length_mm", DoubleType(), True),
        StructField("bill_depth_mm", DoubleType(), True),
        StructField("flipper_length_mm", DoubleType(), True),
        StructField("body_mass_g", DoubleType(), True),
        StructField("sex", StringType(), True),
    ]
)


In [37]:
data = spark.createDataFrame(data=pd_data, schema=schema)
data = data.replace(float("nan"), None)

data.printSchema()
data.show()


root
 |-- species: string (nullable = true)
 |-- island: string (nullable = true)
 |-- bill_length_mm: double (nullable = true)
 |-- bill_depth_mm: double (nullable = true)
 |-- flipper_length_mm: double (nullable = true)
 |-- body_mass_g: double (nullable = true)
 |-- sex: string (nullable = true)

+-------+---------+--------------+-------------+-----------------+-----------+------+
|species|   island|bill_length_mm|bill_depth_mm|flipper_length_mm|body_mass_g|   sex|
+-------+---------+--------------+-------------+-----------------+-----------+------+
| Adelie|Torgersen|          39.1|         18.7|            181.0|     3750.0|  Male|
| Adelie|Torgersen|          39.5|         17.4|            186.0|     3800.0|Female|
| Adelie|Torgersen|          40.3|         18.0|            195.0|     3250.0|Female|
| Adelie|Torgersen|          null|         null|             null|       null|  null|
| Adelie|Torgersen|          36.7|         19.3|            193.0|     3450.0|Female|
| Adelie|To

**Exercise 51: Calculate the number of rows of each `species` in `data`**

In [39]:
data.groupBy("species").agg({"species": "count"}).show()


+---------+--------------+
|  species|count(species)|
+---------+--------------+
|   Adelie|           152|
|Chinstrap|            68|
|   Gentoo|           124|
+---------+--------------+



**Exercise 52: Calculate the sum of `body_mass_g` by `island` in `data`**

In [42]:
data.groupBy("island").agg({"body_mass_g": "sum"}).show()


+---------+----------------+
|   island|sum(body_mass_g)|
+---------+----------------+
|    Dream|        460400.0|
|Torgersen|        189025.0|
|   Biscoe|        787575.0|
+---------+----------------+



**Exercise 53: Calculate the mean of `body_mass_g` by non-null `sex` in `data`**

In [46]:
data.filter("sex IS NOT NULL").groupBy("sex").agg({"body_mass_g": "mean"}).show()


+------+------------------+
|   sex|  avg(body_mass_g)|
+------+------------------+
|Female|3862.2727272727275|
|  Male| 4545.684523809524|
+------+------------------+



**Exercise 54: Calculate the maximum and minimum of `flipper_length_mm` together by `species` in `data`**

In [50]:
data.groupBy("species").agg(
    F.min("flipper_length_mm"), F.max("flipper_length_mm")
).show()


+---------+----------------------+----------------------+
|  species|min(flipper_length_mm)|max(flipper_length_mm)|
+---------+----------------------+----------------------+
|   Adelie|                 172.0|                 210.0|
|Chinstrap|                 178.0|                 212.0|
|   Gentoo|                 203.0|                 231.0|
+---------+----------------------+----------------------+



**Exercise 55: Calculate the median of `body_mass_g` by `species` and `sex` in `data`**

In [56]:
data.filter("sex IS NOT NULL").groupBy("species", "sex").agg(
    F.expr("percentile_approx(body_mass_g, 0.5)").alias("median(body_mass_g)")
).show()


+---------+------+-------------------+
|  species|   sex|median(body_mass_g)|
+---------+------+-------------------+
|   Adelie|Female|             3400.0|
|   Adelie|  Male|             4000.0|
|Chinstrap|Female|             3525.0|
|Chinstrap|  Male|             3950.0|
|   Gentoo|  Male|             5500.0|
|   Gentoo|Female|             4700.0|
+---------+------+-------------------+



**Exercise 56: Calculate the standard deviation of `bill_depth_mm` in cm instead of mm by `island` in `data`**

In [61]:
data.groupBy("island").agg(F.stddev("bill_depth_mm").alias("std")).withColumn(
    "std", F.col("std") / 10
).show()


+---------+-------------------+
|   island|                std|
+---------+-------------------+
|    Dream|0.11331158867932375|
|Torgersen|0.13394467690277295|
|   Biscoe|0.18207213770581204|
+---------+-------------------+



**Exercise 57: Calculate the covariance between `flipper_length_mm` and `body_mass_g` by `species` and `island` in `data`**

In [62]:
data.groupBy("species", "island").agg(
    F.covar_samp("flipper_length_mm", "body_mass_g")
).show()


+---------+---------+------------------------------------------+
|  species|   island|covar_samp(flipper_length_mm, body_mass_g)|
+---------+---------+------------------------------------------+
|   Adelie|   Biscoe|                         1727.021670190277|
|   Adelie|Torgersen|                        1209.2254901960778|
|   Adelie|    Dream|                        1378.6525974025978|
|Chinstrap|    Dream|                          1758.53819139596|
|   Gentoo|   Biscoe|                        2297.1444755431153|
+---------+---------+------------------------------------------+



**Exercise 58: Calculate the absolute correlation between `bill_length_mm` and `bill_depth_mm` by `species` and `sex` in `data`**

In [64]:
data.filter("sex IS NOT NULL").groupBy("species", "sex").agg(
    F.abs(F.corr("bill_length_mm", "bill_depth_mm"))
).show()


+---------+------+----------------------------------------+
|  species|   sex|abs(corr(bill_length_mm, bill_depth_mm))|
+---------+------+----------------------------------------+
|   Adelie|Female|                     0.16063614883268784|
|   Adelie|  Male|                      0.0382471561845751|
|Chinstrap|Female|                     0.25631708027284544|
|Chinstrap|  Male|                      0.4462703251603263|
|   Gentoo|  Male|                      0.3067674738646834|
|   Gentoo|Female|                     0.43044403942836673|
+---------+------+----------------------------------------+



**Exercise 59: Calculate the number of null values in `sex` by `island` in `data`**

In [70]:
data.groupBy("island").agg(
    F.count(F.when(F.col("sex").isNull(), "sex")).alias("null_sex_count")
).show()


+---------+--------------+
|   island|null_sex_count|
+---------+--------------+
|    Dream|             1|
|Torgersen|             5|
|   Biscoe|             5|
+---------+--------------+



**Exercise 60: Sort `data` on `bill_depth_mm` grouped by `sex` and assign it to `data_gs`**

In [11]:
data_gs = data.groupby("sex").apply(lambda x: x.sort_values("bill_depth_mm"))
data_gs


species     island  bill_length_mm  bill_depth_mm  \
sex                                                            
Female 244  Gentoo     Biscoe            42.9           13.1   
       220  Gentoo     Biscoe            46.1           13.2   
       268  Gentoo     Biscoe            44.9           13.3   
       228  Gentoo     Biscoe            43.3           13.4   
       225  Gentoo     Biscoe            46.5           13.5   
...            ...        ...             ...            ...   
Male   35   Adelie      Dream            39.2           21.1   
       61   Adelie     Biscoe            41.3           21.1   
       49   Adelie      Dream            42.3           21.2   
       13   Adelie  Torgersen            38.6           21.2   
       19   Adelie  Torgersen            46.0           21.5   

            flipper_length_mm  body_mass_g     sex  
sex                                                 
Female 244              215.0       5000.0  Female  
       220              211.0       4500.0  Female  
       268              213.0       5100.0  Female  
       228              209.0       4400.0  Female  
       225              210.0       4550.0  Female  
...                       ...          ...     ...  
Male   35               196.0       4150.0    Male  
       61               195.0       4400.0    Male  
       49               191.0       4150.0    Male  
       13               191.0       3800.0    Male  
       19               194.0       4200.0    Male  

[333 rows x 7 columns]

✅ This completes Set 6: Grouping Methods (Exercises 51-60)

Original exercises for Datatable package can be found [here](https://github.com/vopani/datatableton)